In [1]:
import glob
import sys
import numpy as np
import pandas as pd
import os
import time
import argparse
from gensim.models import Word2Vec
import pandas as pd
import argparse
import scipy.sparse as sp
# import dgl
import torch
import pickle
from math import log

# from utils import *
from sklearn.feature_extraction.text import CountVectorizer

print(os.getcwd())

/Users/amy/py/proj3


In [2]:
def load_quadruples(inPath, fileName, fileName2=None, fileName3=None):
    with open(os.path.join(inPath, fileName), 'r') as fr:
        quadrupleList = []
        times = set()
        for line in fr:
            line_split = line.split()
            head = int(line_split[0])
            tail = int(line_split[2])
            rel = int(line_split[1])
            time = int(line_split[3])
            quadrupleList.append([head, rel, tail, time])
            times.add(time)
        # times = list(times)
        # times.sort()
    if fileName2 is not None:
        with open(os.path.join(inPath, fileName2), 'r') as fr:
            for line in fr:
                line_split = line.split()
                head = int(line_split[0])
                tail = int(line_split[2])
                rel = int(line_split[1])
                time = int(line_split[3])
                quadrupleList.append([head, rel, tail, time])
                times.add(time)

    if fileName3 is not None:
        with open(os.path.join(inPath, fileName3), 'r') as fr:
            for line in fr:
                line_split = line.split()
                head = int(line_split[0])
                tail = int(line_split[2])
                rel = int(line_split[1])
                time = int(line_split[3])
                quadrupleList.append([head, rel, tail, time])
                times.add(time)
    times = list(times)
    times.sort()

    return np.asarray(quadrupleList), np.asarray(times)


In [3]:
dp = 'data/'
dn = 'india'
def get_raw_text_l(dp, dn):
    text_f = os.path.join(dp, dn, 'text.txt')
    with open(text_f, 'r') as f:
        text_l = f.read().splitlines()
    # text_l = [text.split(',') for text in text_l]
    return np.array(text_l)

def get_entity(dp, dn):
    ent_f = os.path.join(dp, dn, 'entity2id.txt')
    ent_l = pd.read_csv(ent_f, sep='\t').values
    # with open(ent_f, 'r') as f:
    #     ent_l = f.read().splitlines()
    ent_l = ent_l[:,0]
    return ent_l # array

In [4]:
def count_ent_in_text(dp, dn):
    ent_l = get_entity(dp, dn)
    text_l = get_raw_text_l(dp, dn)
    dict = {}
    for t in text_l[:100]:
        # print(t)
        for e in ent_l:
            if e in t:
                if dict.get(e):
                    dict[e] = dict.get(e)+1
                else:
                    dict[e] = 1
    
    print(dict)
    return dict

In [5]:
dict = count_ent_in_text(dp, dn)

{'India': 27, 'Pakistan': 13, 'RIA Novosti': 2, 'Parliament': 3, 'Telugu Desam Party': 2, 'Australia': 3, 'Canada': 3, 'Pranab Mukherjee': 1, 'Xinhua': 3, 'Bangladesh': 3, 'Khaleda Zia': 2, 'Open University': 2, 'Ashok Gehlot': 2, 'Nitish Kumar': 2, 'Bharatiya Janata': 1, 'Central Bureau of Investigation': 1, 'Janata Party': 1, 'Sajjan Kumar': 1, 'Janata': 1, 'Taliban': 5, 'Tarun Gogoi': 1, 'Nepal': 1, 'Bara': 2, 'S.P. Singh': 1, 'S.M. Krishna': 2, 'Usha Singh': 1, 'Vai': 1, 'Russia': 1, 'Afghanistan': 1, 'Israel': 1, 'Yemen': 1, 'Fahmida Mirza': 1}


In [7]:
dict.keys()

dict_keys(['India', 'Pakistan', 'RIA Novosti', 'Parliament', 'Telugu Desam Party', 'Australia', 'Canada', 'Pranab Mukherjee', 'Xinhua', 'Bangladesh', 'Khaleda Zia', 'Open University', 'Ashok Gehlot', 'Nitish Kumar', 'Bharatiya Janata', 'Central Bureau of Investigation', 'Janata Party', 'Sajjan Kumar', 'Janata', 'Taliban', 'Tarun Gogoi', 'Nepal', 'Bara', 'S.P. Singh', 'S.M. Krishna', 'Usha Singh', 'Vai', 'Russia', 'Afghanistan', 'Israel', 'Yemen', 'Fahmida Mirza'])

In [19]:
# check entities in Test sets, if they exist in training
# train_f = os.path.join(dp, dn, 'train.txt')
# valid_f = os.path.join(dp, dn, 'valid.txt')
# test_f = os.path.join(dp, dn, 'test.txt')
inPath = os.path.join(dp, dn)
train_data, train_times = load_quadruples(inPath, 'train.txt')
dev_data, dev_times = load_quadruples(inPath, 'valid.txt')
test_data, test_times = load_quadruples(inPath, 'test.txt')

In [57]:
src, rel, dst, _ = train_data.transpose()
train_entities = np.concatenate((src,dst))
train_entities = list(set(train_entities))
print('train',len(train_entities))

train 5432


In [56]:
src, rel, dst, _ = dev_data.transpose()
dev_entities = np.concatenate((src,dst))
dev_entities = list(set(dev_entities))
print('dev',len(dev_entities))

dev 2253


In [58]:
src, rel, dst, _ = test_data.transpose()
test_entities = np.concatenate((src,dst))
test_entities = list(set(test_entities))
print('test',len(test_entities))

test 2482


In [11]:
ent_l = get_entity(dp, dn)
print('all',len(ent_l))

ent_l_filter = []
for n in ent_l:
    idx=n.find('(')
    if idx >= 0:
        n = n[:idx-1]
    ent_l_filter.append(n)

print(ent_l_filter[:20],len(ent_l_filter),len(set(ent_l_filter)))


all 6297
['India', 'Citizen', 'Bharatiya Janata', 'Narendra Modi', 'Ministry', 'Member of the Judiciary', 'Cabinet / Council of Ministers / Advisors', 'Congress', 'Government', 'Manmohan Singh', 'Head of Government', 'Court Judge', 'Criminal', 'Activist', 'Military', 'Protester', 'Student', 'Barack Obama', 'Rahul Gandhi', 'Employee'] 6297 3682


In [12]:
set(ent_l_filter)

{'Toshimitsu Motegi',
 'L.V. Singh',
 'V.K. Saraswat',
 'Reporters Sans Frontieres',
 'University of Tehran',
 'Defense Attorney',
 'Tenzin Gyatso',
 'Pepsico',
 'Engineer',
 'Special Commission',
 'INA',
 'Judd Gregg',
 'Abdul Salam Zaeef',
 'Al Qaeda',
 'Emirates Airlines',
 'Harjit Singh',
 'Alur',
 'Brijender Singh',
 'Archbishop',
 'Kirpal Singh',
 'D Company',
 'Arnold Schwarzenegger',
 'Amar Sinha',
 'Human Rights Watch',
 'Angola',
 'Hamid Raja',
 'Civil Defence Forces',
 'Azam Khan',
 'Penny Pritzker',
 'Rishad Bathiudeen',
 'Uzi Landau',
 'Chief Minister Yadav',
 'Albert del Rosario',
 'Vietnam Fatherland Front',
 'Robert James Brown',
 'Kamsá',
 'Edward Nalbandyan',
 'Rizwan Akhtar',
 'International Labor Organization',
 'Ahmed Adeeb',
 'Veerabhadra Rao',
 'Honorary Consul',
 'Danilo Turk',
 'Keith B. Alexander',
 'UiD',
 'Sadiq Khan',
 'Paraguay',
 'Rioter',
 'Babulal Marandi',
 'Islamic Preacher',
 'Iranian Oil Company',
 'Maumoon Abdul Gayoom',
 'Mushahid Hussain Sayed',


In [60]:
# entity in test and not in train
tl = []
for e in test_entities:
    if e not in train_entities:
        tl.append(e)
print(len(tl))

# entity in dev and not in train
dl = []
for e in dev_entities:
    if e not in train_entities:
        dl.append(e)
print(len(dl))

530
402


In [74]:
tl_name = []
for i in tl:
    tl_name.append(ent_l[i])
print(tl_name[:10])

dl_name = []
for i in dl:
    dl_name.append(ent_l[i])
print(dl_name[:10])

['Left Party (India)', 'Military (Sri Lanka)', 'Interpol', 'Arab League', 'Kumari Selja', 'Nandan Nilekani', 'Aziz Qureshi', 'Ministry (Sri Lanka)', 'Congress (United States)', 'Khawaja Asif']
['Left Party (India)', 'Military (Sri Lanka)', 'Interpol', 'Arab League', 'Labour Party', 'Ministry (Sri Lanka)', 'Khawaja Asif', 'Haider Ali', 'Gautam Adani', 'Ashfaq Pervez Kayani']


In [66]:
# 318471  0-318470
text_l = get_raw_text_l(dp, dn)
text_l_train = text_l[:318471]
print(len(text_l_train))

318471


In [68]:
dict = {}
for t in text_l_train:
    # print(t)
    for e in tl_name:
        if e in t:
            if dict.get(e):
                dict[e] = dict.get(e)+1
            else:
                dict[e] = 1

print(dict)
len(dict)

{'Judd Gregg': 2, 'British Airways': 2, 'Minister Parkash': 148, 'Botswana': 15, 'Chung Mong-koo': 3, 'Kumari Selja': 51, 'Vinod Sharma': 10, 'Man Singh': 18, 'Sunday Times': 12, 'Kasit Piromya': 3, 'EAS': 85, 'Andrew Smith': 1, 'Kumar Sharma': 67, 'United Aircraft Corporation': 3, 'Muslim Brotherhood': 33, 'Arab League': 86, 'Shura Council': 15, 'Interpol': 96, 'Gilbert Houngbo': 5, 'Ivorian Popular Front': 4, 'State Peace and Development': 29, 'General Vij': 14, 'Abbas El Fassi': 2, 'Costa Rica': 3, 'The Frontier Post': 4, 'Sherry Rehman': 11, 'Arya Samaj': 14, 'Sri Lanka Freedom Party': 1, 'Pak Ui Chun': 4, 'Angus Houston': 2, 'ISAR': 21, 'Bhaskara Rao': 4, 'Ed Miliband': 4, 'Kamal Hossain': 1, 'Lithuania': 20, 'ROKN': 2, 'Juan Carlos I': 17, 'Charles de Gaulle': 4, 'Bogdan Borusewicz': 13, 'Renmin Ribao': 6, 'Hoshyar Zebari': 3, 'Desmond Tutu': 1, 'Abdul Gafoor': 1, 'Financial Action Task Force on Money Laundering': 1, 'Deepak Rai': 3, 'Zabihollah Mojahed': 4, 'Seiji Maehara': 4, '

124

In [72]:
len(list(dict.keys()))

124

In [75]:
dict_dev = {}
for t in text_l_train:
    # print(t)
    for e in dl_name:
        if e in t:
            if dict_dev.get(e):
                dict_dev[e] = dict_dev.get(e)+1
            else:
                dict_dev[e] = 1

print(dict_dev)

{'Vai': 495, 'Judd Gregg': 2, 'Niger': 397, 'Ma Xiaotian': 12, 'Birendra': 13, 'Botswana': 15, 'Social Democratic Party': 26, 'National Democratic Front': 188, 'Jean-Max Bellerive': 2, 'Vinod Sharma': 10, 'Hukum Singh': 8, 'Bernard Shamlaye': 2, 'Samoa': 18, 'Arab League': 86, 'Interpol': 96, 'Ashfaq Pervez Kayani': 24, 'Kanti Singh': 5, 'Charles Ble Goude': 4, 'Norodom Sihamoni': 4, 'Abbas El Fassi': 2, 'United National Front': 2, 'Wu Bangguo': 41, 'Avigdor Lieberman': 14, 'Arya Samaj': 14, 'Jai Kumar': 6, 'Campbell Newman': 4, 'Riaz Khokhar': 1, 'S.A. Khan': 2, 'Madan Bafna': 2, 'Guido Westerwelle': 20, 'Motorola': 2, 'European Aeronautic Defence and Space Company': 4, 'Yuval Steinitz': 4, 'UAE Armed Forces': 1, 'Zhu Rongji': 2, 'Jaffer Sharief': 6, 'Radio France Internationale': 15, 'H.N. Singh': 1, 'Labour Party': 9, 'Ghulam Mohammad Bhat': 5, 'G7': 2, 'Rabobank': 1, 'Wildlife Trust': 1, 'Mohammed Shafi': 9, 'Lieutenant Governor Singh': 1, 'Bangladesh Bank': 1, 'Milan Stech': 3, 'R

In [76]:
len(list(dict_dev.keys()))

103

In [78]:
tl_name

['Left Party (India)',
 'Military (Sri Lanka)',
 'Interpol',
 'Arab League',
 'Kumari Selja',
 'Nandan Nilekani',
 'Aziz Qureshi',
 'Ministry (Sri Lanka)',
 'Congress (United States)',
 'Khawaja Asif',
 'Civil Aviation Department (India)',
 'Department of Defense',
 'Rajeev Kher',
 'Gautam Adani',
 'Foreign Affairs (Bahrain)',
 'Minister Nemwang',
 'Ebrahim Rahimpour',
 'Centre for Science and Environment',
 'Abdul Subhan',
 'A.H. Desai',
 'Activist (Greenpeace)',
 'Citizen (Denmark)',
 'Head of Government (Australia)',
 'Arya Samaj',
 'Business (Taiwan)',
 'Combatant (Yemen)',
 'Vinod Sharma',
 'Caribbean Media Corporation',
 'Government (Switzerland)',
 'Financial Action Task Force on Money Laundering',
 'Militant (Afghanistan)',
 'Kostyantyn Gryshchenko',
 'Sikh (Pakistan)',
 'Ernest Bai Koroma',
 'Chung Mong-koo',
 'Center for Human Rights (India)',
 'Finance / Economy / Commerce / Trade Ministry (Australia)',
 'Brahmin (India)',
 'S.S. Chauhan',
 'Tharman Shanmugaratnam',
 'Hunter

In [89]:
tl_name_filter = []
for n in tl_name:
    idx=n.find('(')
    if idx >= 0:
        n = n[:idx-1]
    tl_name_filter.append(n)

print(tl_name_filter,len(tl_name_filter))


['Left Party', 'Military', 'Interpol', 'Arab League', 'Kumari Selja', 'Nandan Nilekani', 'Aziz Qureshi', 'Ministry', 'Congress', 'Khawaja Asif', 'Civil Aviation Department', 'Department of Defense', 'Rajeev Kher', 'Gautam Adani', 'Foreign Affairs', 'Minister Nemwang', 'Ebrahim Rahimpour', 'Centre for Science and Environment', 'Abdul Subhan', 'A.H. Desai', 'Activist', 'Citizen', 'Head of Government', 'Arya Samaj', 'Business', 'Combatant', 'Vinod Sharma', 'Caribbean Media Corporation', 'Government', 'Financial Action Task Force on Money Laundering', 'Militant', 'Kostyantyn Gryshchenko', 'Sikh', 'Ernest Bai Koroma', 'Chung Mong-koo', 'Center for Human Rights', 'Finance / Economy / Commerce / Trade Ministry', 'Brahmin', 'S.S. Chauhan', 'Tharman Shanmugaratnam', 'Hunter', 'Gusharan Singh', 'Opposition Activist', 'Sipah-e-Sahaba Pakistan', 'Carlos Agostinho do Rosário', 'Member of Parliament', 'Armed Rebel', 'Mukhtiar Singh', 'Head of Government', 'Army Personnel', 'Government', 'Oil Tanker'

In [93]:
dict = {}
for t in text_l_train:
    # print(t)
    for e in tl_name_filter:
        if e in t:
            if dict.get(e):
                dict[e] = dict.get(e)+1
            else:
                dict[e] = 1

print(dict)
keys = list(dict.keys())

{'Police': 117606, 'Ministry': 21368, 'Human Rights Commission': 377, 'Congress': 51616, 'Sikh': 1102, 'Government': 48010, 'Director General': 588, 'Muslim': 3072, 'Communist Party': 1574, 'Tamil': 3569, 'Military': 1084, 'Governor': 2103, 'Education': 2306, 'High Commission': 815, 'Student': 4730, 'Judd Gregg': 2, 'Buddhist': 182, 'Media': 1308, 'Activist': 4140, 'British Airways': 2, 'Hindu': 4138, 'Children': 302, 'Industry': 1758, 'Minister Parkash': 148, 'Women': 3591, 'Criminal': 1878, 'Militant': 1332, 'Men': 6430, 'Botswana': 15, 'Daily Star': 56, 'Admiral': 410, 'Business': 1056, 'Bank': 1302, 'Jew': 256, 'Envoy': 194, 'Terrorist': 435, 'Protester': 930, 'Scientist': 17, 'Chung Mong-koo': 3, 'Secretariat': 335, 'Member of Parliament': 1148, 'Airline': 93, 'Kumari Selja': 51, 'Member of Legislative': 24, 'Public Prosecutor': 124, 'Employee': 525, 'Vinod Sharma': 10, 'Foreign Affairs': 2608, 'Man Singh': 18, 'Main Opposition': 4, 'Sunday Times': 12, 'Kasit Piromya': 3, 'EAS': 8

In [91]:
dl_name_filter = []
for n in dl_name:
    idx=n.find('(')
    if idx >= 0:
        n = n[:idx-1]
    dl_name_filter.append(n)

print(dl_name_filter[:20],len(dl_name_filter))
 

['Left Party', 'Military', 'Interpol', 'Arab League', 'Labour Party', 'Ministry', 'Khawaja Asif', 'Haider Ali', 'Gautam Adani', 'Ashfaq Pervez Kayani', 'Foreign Affairs', 'Ebrahim Rahimpour', 'Police', 'Guido Westerwelle', 'Oldemiro Julio Baloi', 'Army Personnel', 'A.H. Desai', 'Activist', 'Head of Government', 'Avigdor Lieberman'] 402


In [116]:
dict_dev = {}
for t in text_l_train: # text_l_train
    # print(t)
    for e in dl_name_filter:
        if e in t:
            if dict_dev.get(e):
                dict_dev[e] = dict_dev.get(e)+1
            else:
                dict_dev[e] = 1

print(len(dict_dev))
keys_dev = list(dict_dev.keys())

167


In [108]:
non_ents = []
for n in dl_name_filter:
    if n not in keys_dev:
        non_ents.append(n)
#     else:
#         print(dict_dev[n])
len(dl_name_filter),len(non_ents)
# non_ents

(402, 163)

In [101]:
len(dl_name),len(dl_name_filter),len(keys_dev),len(non_ents)

(402, 402, 167, 163)

In [113]:
# dict_dev

In [114]:
print(len(list(set(dl_name_filter))))
print(len(list(set(tl_name_filter))))

307
380


In [120]:
len(list(set(non_ents))),list(set(non_ents))

(140,
 ['Army Personnel',
  'Yuli Edelstein',
  'M.S. Calcutta',
  'Other Authorities / Officials',
  'Khawaja Asif',
  'Roger B. Wilson',
  'D Company',
  'Rebel Commander',
  'Armed Rebel',
  'Bank Mandiri',
  'Armed Separatist',
  'Preacher',
  'Anabel González',
  'Intelligence Chief',
  'Democratic Alliance Party',
  'Patali Champika Ranawaka',
  'Energy Department/Ministry',
  'J.P. Mathur',
  'Avijit Roy',
  'Order and Justice',
  'Sihasak Phuangketkeow',
  'Stanley Fischer',
  'Liberty Party',
  'Aviation Minister Ibrahim',
  'Swraj Paul',
  'Senate of Pakistan',
  'Maria da Luz Guebuza',
  'Wunnakyawhtin Win Myint',
  'Women / Children / Social / Welfare / Development / Religion Ministry',
  'Displaced Families',
  'Glaxosmithkline',
  'Ansar Bayt Al-Maqdis',
  'Awad Ahmed al-Jaz',
  'Government Official',
  'Media Personnel',
  'Libération',
  'Keshab Gogoi',
  'Commonwealth Secretariat',
  'Socialist Front',
  'Shamser Mobin Chowdhury',
  'Ram Chandra Yadav',
  'Deputy Inspe

In [121]:
import math

def sigmoid(x):
    return 1 / (1 + math.exp(-x))

In [132]:
sigmoid(0.0)

0.5

In [137]:
np.tanh(0.5)

0.46211715726000974

In [138]:
def precision_at_k_sig(true_rank, prob_rank, k_l):
     
    precision_l = []
    for k in k_l:
        precision = len(np.intersect1d(true_rank, prob_rank[:k], return_indices=False)) / (len(prob_rank[:k]) + 1e-10)
        precision_l.append(precision)
    return precision_l

def recall_at_k_sig(true_rank, prob_rank, k_l):
    recall_l = []
    for k in k_l:
        recall = len(np.intersect1d(true_rank, prob_rank[:k], return_indices=False)) / (len(true_rank) + 1e-10)
        recall_l.append(recall)
    return recall_l

In [139]:
true_rank = np.array([1,2,3,4,5,6,7,8,9])
prob_rank = np.array([10,2,3,5,6,2])
k_l = [1,2,3,4,5]

precision_at_k_sig(true_rank, prob_rank, k_l)

[0.0, 0.499999999975, 0.6666666666444444, 0.74999999998125, 0.799999999984]

In [140]:
recall_at_k_sig(true_rank, prob_rank, k_l)

[0.0,
 0.11111111110987654,
 0.22222222221975307,
 0.3333333333296296,
 0.44444444443950615]

In [143]:
[[]]*3

[[], [], []]

In [144]:
k_l[:0]

[]

In [146]:
if not 0.0:
    print('-')

-


In [129]:
a = np.array([0.1,0.2,0.3,0.4,0.0])
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

softmax(a) 

array([0.17917069, 0.19801424, 0.21883958, 0.24185514, 0.16212035])